# **FITS to HDF5**

In [2]:
import numpy as np
import pandas as pd
import h5py
import astropy.table
from astropy.table import Table
import astropy.io.fits as pf

In [3]:
filename = '/home/raulteixeira/scratch-midway2/CosmicShearData/shear_test_coadd_object_gold.fits'
pf.open(filename).info()

Filename: /home/raulteixeira/scratch-midway2/CosmicShearData/shear_test_coadd_object_gold.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    238   18617785R x 45C   [K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, J, J, J, J, I, I, I, I, K, J, J, J, J, 48A]   


In [4]:
data = Table.read('/home/raulteixeira/scratch-midway2/CosmicShearData/shear_test_coadd_object_gold.fits')

In [5]:
#s/g cut Y3
WAVG_SPREAD_MODEL_I = data['WAVG_SPREAD_MODEL_I']
WAVG_SPREADERR_MODEL_I = data['WAVG_SPREADERR_MODEL_I']

s1 = (WAVG_SPREAD_MODEL_I + 3. * WAVG_SPREADERR_MODEL_I) > 0.005
s2 = (WAVG_SPREAD_MODEL_I + 1. * WAVG_SPREADERR_MODEL_I) > 0.003
s3 = (WAVG_SPREAD_MODEL_I - 1. * WAVG_SPREADERR_MODEL_I) > 0.001
ext_wavg = s1.astype(int) + s2.astype(int) + s3.astype(int)
sg_mask = (ext_wavg>=2)

In [ ]:
#s/g cut Y6


In [7]:
data = data[sg_mask]

In [25]:
# creating mask
# mask1 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_I']!=99) * (data['MAG_AUTO_Z']!=99)
# mask2 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_I']!=99) 
# mask3 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_Z']!=99)
# mask4 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_I']!=99) * (data['MAG_AUTO_Z']!=99)
# mask5 = (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_I']!=99) * (data['MAG_AUTO_Z']!=99)

# mask = mask1 + mask2 + mask3 + mask4 + mask5

In [26]:
# data=data[mask]

In [8]:
bandlist = ['G', 'R', 'I', 'Z']
for band in bandlist:
    data[f'MAG_AUTO_{band}'][data[f'MAG_AUTO_{band}']==99]=-99
    data[f'MAGERR_AUTO_{band}'][data[f'MAG_AUTO_{band}']==99]=0

In [9]:
tiles = np.unique(np.array(data['TILENAME']))

In [ ]:
#creating the table to be converted
for tile in np.unique(data['TILENAME']):
    #converting to pandas dataframe
    masktile=data['TILENAME']==tile
    table_i = data[masktile]['COADD_OBJECT_ID', 'TILENAME', 'MAG_AUTO_G', 'MAG_AUTO_R'\
                         , 'MAG_AUTO_I', 'MAG_AUTO_Z', 'MAGERR_AUTO_G', 'MAGERR_AUTO_R'\
                         , 'MAGERR_AUTO_I', 'MAGERR_AUTO_Z'] 
                         #[tab_all['extended_class']>1]
    dframe_i = pd.DataFrame(data=table_i['COADD_OBJECT_ID']\
                            , columns = ['COADD_OBJECT_ID'])
    for label in table_i.columns:
            dframe_i[label] = data[label][masktile]
    dframe_i.to_hdf(f'/home/raulteixeira/scratch-midway2/CosmicShearData/bpztiles/pzinput/pzinput_sg_{tile}.h5', key='df')

In [ ]:
tile = tiles[0].astype('str')

In [14]:
#converting to pandas dataframe
masktile=data['TILENAME']==tile
table_i = data[mask][masktile]['COADD_OBJECT_ID', 'TILENAME', 'MAG_AUTO_G', 'MAG_AUTO_R'\
                     , 'MAG_AUTO_I', 'MAG_AUTO_Z', 'MAGERR_AUTO_G', 'MAGERR_AUTO_R'\
                     , 'MAGERR_AUTO_I', 'MAGERR_AUTO_Z'] 
                     #[tab_all['extended_class']>1]
dframe_i = pd.DataFrame(data=table_i['COADD_OBJECT_ID']\
                        , columns = ['COADD_OBJECT_ID'])
for label in table_i.columns:
        dframe_i[label] = data[label][masktile]
dframe_i.to_hdf('/home/raulteixeira/photoz/CosmicShearData/bpztiles/pzinput/pz_input_gold_'+tile+'.h5', key='df')